# Verificação das hipóteses relacionadas a nota média

## Verificação das hipóteses 0, 1 e 5

Hipótese 0: Se o pedido é cancelado, a nota do pedido é menor \
Hipótese 1: Se o pedido foi entregue com atraso, a nota do pedido será menor \
Hipótese 5: Se o pedido atrasar sua nota será menor que três

### Definição do dataframe

In [1]:
from pyspark.sql import SparkSession, functions as F
spark = SparkSession.builder.getOrCreate()
df_reviews = spark.read \
                 .option('escape', '\"') \
                 .csv('./dataset/olist_order_reviews_dataset.csv', header=True, multiLine=True, inferSchema=True)
df_orders = spark.read \
                 .option('escape', '\"') \
                 .csv('./dataset/olist_orders_dataset.csv', header=True, multiLine=True)
df = df_orders.join(df_reviews, df_orders.order_id == df_reviews.order_id)

In [2]:
df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: string (nullable = true)
 |-- order_approved_at: string (nullable = true)
 |-- order_delivered_carrier_date: string (nullable = true)
 |-- order_delivered_customer_date: string (nullable = true)
 |-- order_estimated_delivery_date: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: integer (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: timestamp (nullable = true)
 |-- review_answer_timestamp: timestamp (nullable = true)



### Calculo nota média geral

In [3]:
df.select(F.mean('review_score')).show()

+-----------------+
|avg(review_score)|
+-----------------+
|          4.07089|
+-----------------+



### Calculo nota média dos cancelados

In [4]:
df_canceled = df.filter(F.col('order_status')=='canceled')

In [5]:
df_canceled.select(F.mean('review_score')).show()

+------------------+
| avg(review_score)|
+------------------+
|1.8108108108108107|
+------------------+



## Calculo nota média dos atrasados

In [6]:
df_late = df.filter(F.col('order_delivered_customer_date') > F.col('order_estimated_delivery_date'))

In [7]:
df_late.select(F.mean('review_score')).show()

+-----------------+
|avg(review_score)|
+-----------------+
|  2.5465293668955|
+-----------------+



### Calculo de pedidos atrasados com nota maior ou igual a 3 (hipótese 5)

In [8]:
print("O numero de pedidos atrasados com nota >=3 é de",df_late.filter(F.col('review_score')>=3).count())

O numero de pedidos atrasados com nota >=3 é de 3572


In [17]:
print("Porcentagem de pedidos atrasados com nota >= 3:",
      round(df_late.filter(F.col('review_score')>=3).count() / df_late.count() * 100,2))

Porcentagem de pedidos atrasados com nota >= 3: 45.41


## Conclusões

### A hipótese 0 é válida, pois a média das notas de todos os pedidos é maior que a média das notas dos pedidos cancelados

### A hipótese 1 é válida, pois a média das notas de todos os pedidos é maior que a média das notas dos pedidos cancelados

### A hipótese 5 é inválida pois existem pedidos atrasados que possuem nota maior ou igual a 3, são 3572 pedidos com essas características